# Notebook to convert raw data to h5ad

**Created by :** Srivalli Kolla

**Created on :** 16 April, 2025

**Modified on :** 16 April, 2025

**University of Würzburg**

Env : scanpy (Python 3.12.2)

# Importing Packages

In [1]:
import anndata as ad
import scanpy as sc
import pandas as pd
import os
import datetime

In [2]:
sc.settings.verbosity = 3
sc.logging.print_versions()

timestamp = datetime.datetime.now().strftime("%d_%m_%y")

-----
anndata     0.11.3
scanpy      1.10.4
-----
Cython                      3.0.12
PIL                         11.1.0
anyio                       NA
arrow                       1.3.0
asttokens                   NA
attr                        25.1.0
attrs                       25.1.0
babel                       2.17.0
certifi                     2025.01.31
charset_normalizer          3.4.1
colorama                    0.4.6
comm                        0.2.2
cycler                      0.12.1
cython                      3.0.12
cython_runtime              NA
dateutil                    2.9.0.post0
debugpy                     1.8.12
decorator                   5.2.1
defusedxml                  0.7.1
executing                   2.1.0
fastjsonschema              NA
fqdn                        NA
h5py                        3.13.0
idna                        3.10
igraph                      0.11.8
ipykernel                   6.29.5
ipywidgets                  8.1.5
isoduration               

/home/gruengroup/srivalli/miniconda3/envs/scanpy/lib/python3.13/site-packages/session_info/main.py:213: UserWarning: The '__version__' attribute is deprecated and will be removed in MarkupSafe 3.1. Use feature detection, or `importlib.metadata.version("markupsafe")`, instead.
  mod_version = _find_version(mod.__version__)


# Data loading

In [3]:
cd 'Github/ACM_sn_2025'

/home/gruengroup/srivalli/Github/ACM_sn_2025


In [4]:
base_dir = "./data/Library2/"

samples = [d for d in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, d))]
samples

['B10_Lib2',
 'B5_Lib2',
 'B7_Lib2',
 'B6_Lib2',
 'B8_Lib2',
 'B9_Lib2',
 'B11_Lib2',
 'B3_Lib2',
 'B4_Lib2']

# Data export

1. Load matrix files
2. Add Sample name by splitting the id by '_' and considering only part before split(B3_Lib2 = B3)
3. Writing h5ad file in output path

In [5]:
for sample in samples:
    matrix_dir = os.path.join(base_dir, sample, "filtered_feature_bc_matrix")

    print(f"Loading matrix from {matrix_dir}...")
    adata = sc.read_10x_mtx(matrix_dir, var_names='gene_symbols', cache=False)
    sample_id = sample.split("_")[0]
    adata.obs['sample'] = sample_id
    
    out_file = os.path.join(base_dir, sample, f"{sample_id}_{timestamp}.h5ad")
    adata.write(out_file)

    print(f"Saved to {out_file}")

Loading matrix from ./data/Library2/B10_Lib2/filtered_feature_bc_matrix...
--> This might be very slow. Consider passing `cache=True`, which enables much faster reading from a cache file.
Saved to ./data/Library2/B10_Lib2/B10_16_04_25.h5ad
Loading matrix from ./data/Library2/B5_Lib2/filtered_feature_bc_matrix...
--> This might be very slow. Consider passing `cache=True`, which enables much faster reading from a cache file.
Saved to ./data/Library2/B5_Lib2/B5_16_04_25.h5ad
Loading matrix from ./data/Library2/B7_Lib2/filtered_feature_bc_matrix...
--> This might be very slow. Consider passing `cache=True`, which enables much faster reading from a cache file.
Saved to ./data/Library2/B7_Lib2/B7_16_04_25.h5ad
Loading matrix from ./data/Library2/B6_Lib2/filtered_feature_bc_matrix...
--> This might be very slow. Consider passing `cache=True`, which enables much faster reading from a cache file.
Saved to ./data/Library2/B6_Lib2/B6_16_04_25.h5ad
Loading matrix from ./data/Library2/B8_Lib2/filte

# Data merging

In [6]:
h5ad_paths = [
    os.path.join(base_dir, d, f)
    for d in os.listdir(base_dir)
    for f in os.listdir(os.path.join(base_dir, d))
    if f.endswith(".h5ad")
]

In [7]:
adatas = [sc.read_h5ad(p) for p in h5ad_paths]

adata_combined = ad.concat(adatas, join='inner', label='sample', keys=[a.obs['sample'][0] for a in adatas])
adata_combined

/tmp/ipykernel_1807086/367787295.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  adata_combined = ad.concat(adatas, join='inner', label='sample', keys=[a.obs['sample'][0] for a in adatas])
/home/gruengroup/srivalli/miniconda3/envs/scanpy/lib/python3.13/site-packages/anndata/_core/anndata.py:1756: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


AnnData object with n_obs × n_vars = 173725 × 32285
    obs: 'sample'

In [8]:
adata_combined.obs

,sample
AAACCCAAGAGGTCGT-1,B10
AAACCCAAGCACAAAT-1,B10
AAACCCAAGCCGTTAT-1,B10
AAACCCAAGCTCTATG-1,B10
AAACCCAAGCTGGCCT-1,B10
...,...
TTTGTTGTCCCGTTGT-1,B4
TTTGTTGTCCGATCGG-1,B4
TTTGTTGTCTAGAACC-1,B4
TTTGTTGTCTAGGCAT-1,B4


In [9]:
adata_combined.var

""
Xkr4
Gm1992
Gm19938
Gm37381
Rp1
...
AC124606.1
AC133095.2
AC133095.1
AC234645.1


# Add metadata

In [10]:
metadata = pd.read_csv("./data/acm_sn_metadata.csv",sep= ',') 
metadata

,sample,Sample_Name,Sex,Genotype,Treatment,Condition,Sample_ID
0,B3,20241021_AC_Pkp2-MCMV_Mouse_Single_Heart_1,male,Ctr,noninf,Ctr_noninf,B3_Ctr_noninf
1,B4,20241021_AC_Pkp2-MCMV_Mouse_Single_Heart_2,male,het_KO,noninf,het_KO_noninf,B4_het_KO_noninf
2,B5,20241021_AC_Pkp2-MCMV_Mouse_Single_Heart_3,male,het_KO,MCMV,het_KO_MCMV,B5_het_KO_MCMV
3,B6,20241021_AC_Pkp2-MCMV_Mouse_Single_Heart_4,male,Ctr,noninf,Ctr_noninf,B6_Ctr_noninf
4,B7,20241021_AC_Pkp2-MCMV_Mouse_Single_Heart_5,male,het_KO,noninf,het_KO_noninf,B7_het_KO_noninf
5,B8,20241021_AC_Pkp2-MCMV_Mouse_Single_Heart_6,male,het_KO,MCMV,het_KO_MCMV,B8_het_KO_MCMV
6,B9,20241021_AC_Pkp2-MCMV_Mouse_Single_Heart_7,male,Ctr,noninf,Ctr_noninf,B9_Ctr_noninf
7,B10,20241021_AC_Pkp2-MCMV_Mouse_Single_Heart_8,male,het_KO,noninf,het_KO_noninf,B10_het_KO_noninf
8,B11,20241021_AC_Pkp2-MCMV_Mouse_Single_Heart_9,male,het_KO,MCMV,het_KO_MCMV,B11_het_KO_MCMV


In [11]:
adata_combined.obs = adata_combined.obs.merge(metadata, how='left', on='sample')
adata_combined

AnnData object with n_obs × n_vars = 173725 × 32285
    obs: 'sample', 'Sample_Name', 'Sex', 'Genotype', 'Treatment', 'Condition', 'Sample_ID'

In [12]:
adata_combined.obs

,sample,Sample_Name,Sex,Genotype,Treatment,Condition,Sample_ID
0,B10,20241021_AC_Pkp2-MCMV_Mouse_Single_Heart_8,male,het_KO,noninf,het_KO_noninf,B10_het_KO_noninf
1,B10,20241021_AC_Pkp2-MCMV_Mouse_Single_Heart_8,male,het_KO,noninf,het_KO_noninf,B10_het_KO_noninf
2,B10,20241021_AC_Pkp2-MCMV_Mouse_Single_Heart_8,male,het_KO,noninf,het_KO_noninf,B10_het_KO_noninf
3,B10,20241021_AC_Pkp2-MCMV_Mouse_Single_Heart_8,male,het_KO,noninf,het_KO_noninf,B10_het_KO_noninf
4,B10,20241021_AC_Pkp2-MCMV_Mouse_Single_Heart_8,male,het_KO,noninf,het_KO_noninf,B10_het_KO_noninf
...,...,...,...,...,...,...,...
173720,B4,20241021_AC_Pkp2-MCMV_Mouse_Single_Heart_2,male,het_KO,noninf,het_KO_noninf,B4_het_KO_noninf
173721,B4,20241021_AC_Pkp2-MCMV_Mouse_Single_Heart_2,male,het_KO,noninf,het_KO_noninf,B4_het_KO_noninf
173722,B4,20241021_AC_Pkp2-MCMV_Mouse_Single_Heart_2,male,het_KO,noninf,het_KO_noninf,B4_het_KO_noninf
173723,B4,20241021_AC_Pkp2-MCMV_Mouse_Single_Heart_2,male,het_KO,noninf,het_KO_noninf,B4_het_KO_noninf


In [13]:
combined_path = os.path.join(base_dir, f"acm_lib2_concat_raw_{timestamp}.h5ad")
adata_combined.write(combined_path)